In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
df=pd.read_csv("/kaggle/input/hotel-booking-demand/hotel_bookings.csv")
df.head()

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.drop('company',axis=1,inplace=True)


In [ ]:
fig,axes= plt.subplots(1,1,figsize=(12,9))
sns.heatmap(df.corr())
plt.show()

In [ ]:
df.corr()

In [ ]:
def time_con(x):
    return datetime.datetime.strptime(x,'%Y-%m-%d').date()

df['reservation_status_date'] = df['reservation_status_date'].apply(lambda x : time_con(x))

In [ ]:
(df.reservation_status_date.apply(lambda x : x.year)).value_counts().plot(kind='bar')
plt.show()

In [ ]:
sns.barplot(x="hotel", y="is_canceled", data=df.groupby("hotel").is_canceled.count().reset_index())
plt.ylabel ('count')
plt.show()

In [ ]:
sns.barplot(x="hotel", y="lead_time", hue="is_canceled", data=df.groupby(["hotel","is_canceled"]).lead_time.count().reset_index())
plt.ylabel('count')
plt.show()

In [ ]:
sns.countplot(x="reservation_status", data = df)
plt.show()

In [ ]:
df.drop('agent',axis=1,inplace=True)

In [ ]:
df.drop('hotel',axis=1,inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.fillna(df.mode().iloc[0],inplace=True)

In [ ]:
df.info()

In [ ]:
df.drop('reservation_status_date',inplace=True,axis=1)

In [ ]:
df.isnull().sum()

In [ ]:
X = df.iloc[:,1:]
y = df.iloc[:,0]


In [ ]:
df.dtypes

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

In [ ]:
ct = make_column_transformer(
    (OneHotEncoder(),['meal','distribution_channel','reservation_status','country','arrival_date_month','market_segment','deposit_type','customer_type', 'reserved_room_type','assigned_room_type' ]), remainder = 'passthrough'
    )

In [ ]:
df.dtypes

In [ ]:
X = ct.fit_transform(X).toarray()


In [ ]:
X

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 100)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
explained_variance = pca.explained_variance_ratio_

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0, max_iter=1000)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)